## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-27-10-46-20 +0000,bbc,Prisons begin extra checks after migrant relea...,https://www.bbc.com/news/articles/c70jrrgjp9xo...
1,2025-10-27-10-45-00 +0000,wsj,Huntington Reaches $7.4 Billion Deal to Buy Ca...,https://www.wsj.com/finance/banking/huntington...
2,2025-10-27-10-30-00 +0000,latimes,Shaikin: What are the motives behind Frank McC...,https://www.latimes.com/sports/dodgers/story/2...
3,2025-10-27-10-29-42 +0000,nyt,Javier Milei’s Party Triumphs in Argentina Mid...,https://www.nytimes.com/2025/10/26/world/ameri...
4,2025-10-27-10-29-00 +0000,wsj,"Amazon Pledges More Than $1.6 Billion for AWS,...",https://www.wsj.com/tech/amazon-pledges-more-t...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2330/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
77,trump,38
163,china,15
10,deal,10
75,new,10
168,out,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
23,2025-10-27-09-37-17 +0000,nypost,Trump rules out running as Vance’s VP to gain ...,https://nypost.com/2025/10/27/us-news/trump-ru...,113
76,2025-10-27-04-17-00 +0000,wsj,"Asian Markets Rise as U.S.-China Talks, Trump ...",https://www.wsj.com/finance/stocks/asian-marke...,89
93,2025-10-27-02-01-40 +0000,bbc,US and China agree framework of trade deal ahe...,https://www.bbc.com/news/articles/c1lqj5lz4geo...,85
77,2025-10-27-04-09-01 +0000,nypost,"US, China talks sketch out rare earths, tariff...",https://nypost.com/2025/10/27/world-news/us-ch...,81
212,2025-10-26-12-24-18 +0000,nypost,"US, China reach deal to allow TikTok sale, Bes...",https://nypost.com/2025/10/26/us-news/us-china...,80


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
23,113,2025-10-27-09-37-17 +0000,nypost,Trump rules out running as Vance’s VP to gain ...,https://nypost.com/2025/10/27/us-news/trump-ru...
76,51,2025-10-27-04-17-00 +0000,wsj,"Asian Markets Rise as U.S.-China Talks, Trump ...",https://www.wsj.com/finance/stocks/asian-marke...
14,37,2025-10-27-10-00-00 +0000,nypost,Even Bernie Sanders admits Mamdani won’t have ...,https://nypost.com/2025/10/27/us-news/even-ber...
190,32,2025-10-26-15-13-13 +0000,nypost,Sen. Chris Murphy suggests Dems could end shut...,https://nypost.com/2025/10/26/us-news/sen-chri...
24,31,2025-10-27-09-33-16 +0000,nypost,Frightening video shows military plane nearly ...,https://nypost.com/2025/10/27/us-news/video-sh...
102,30,2025-10-27-00-59-51 +0000,nypost,"US Navy Sea Hawk helicopter, F/A-18F Super Hor...",https://nypost.com/2025/10/26/world-news/us-na...
110,28,2025-10-26-23-18-16 +0000,nypost,Zohran Mamdani insists NYPD Commish Jessica Ti...,https://nypost.com/2025/10/26/us-news/mamdani-...
50,27,2025-10-27-09-00-00 +0000,wsj,Democratic leaders have pinned their hopes on ...,https://www.wsj.com/politics/elections/these-t...
145,27,2025-10-26-20-12-23 +0000,nypost,Gavin Newsom’s story about ‘hustling’ to pay t...,https://nypost.com/2025/10/26/us-news/gavin-ne...
148,25,2025-10-26-19-51-18 +0000,nyt,Hurricane Melissa Could Hit as One of Jamaica’...,https://www.nytimes.com/2025/10/26/weather/jam...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
